In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install snntorch

In [3]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import matthews_corrcoef
from torchvision import models
import snntorch as snn
from snntorch import surrogate
from snntorch import utils
import gc
import matplotlib.pyplot as plt
# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "base_dir": "/content/drive/MyDrive/glacier/Train",
    "project_dir": "/content/drive/MyDrive/Glacier_SNN_EffNetV2_Fixed",

    "model_type": "CNN",
    "remove_se": True,     # Removing SE is critical for SNN stability

    "time_steps": 6,
    "batch_size": 2,
    "lr": 1e-4,
    "epochs": 40,
    "beta": 0.9,
    "threshold": 0.5,
    "slope": 25,
    "num_workers": 2,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

os.makedirs(CONFIG['project_dir'], exist_ok=True)
torch.cuda.empty_cache()
gc.collect()

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)

# ==========================================
# 2. DATASET
# ==========================================
class GlacierDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = Path(base_dir)
        self.band_dirs = [self.base_dir / f"Band{i}" for i in range(1, 6)]
        self.label_dir = self.base_dir / "labels"

        if not self.band_dirs[0].exists(): raise FileNotFoundError("Check Drive Path!")
        self.ids = sorted([p.stem for p in self.band_dirs[0].glob("*.tif")])
        self.transform = transform

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        bands = [cv2.imread(str(d / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED).astype(np.float32) for d in self.band_dirs]
        image = np.stack(bands, axis=-1)
        label = cv2.imread(str(self.label_dir / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED)
        if label.ndim == 3: label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)

        p02, p98 = np.percentile(image, 2), np.percentile(image, 98)
        image = np.clip(image, p02, p98)
        image = (image - image.min()) / (image.max() - image.min() + 1e-6)

        mask = np.zeros_like(label, dtype=np.int64)
        mask[label == 85] = 1; mask[label == 170] = 2; mask[label == 255] = 3

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            return aug["image"].float(), aug["mask"].long()
        return torch.tensor(image.transpose(2,0,1)).float(), torch.tensor(mask).long()

class Wrapper(Dataset):
    def __init__(self, ds, t): self.ds, self.t = ds, t
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        img, mask = self.ds[i]
        img = img.numpy().transpose(1,2,0); mask = mask.numpy()
        res = self.t(image=img, mask=mask)
        return res['image'], res['mask'].long()

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5),
    A.GridDistortion(p=0.3),
    ToTensorV2(),
])
val_transform = A.Compose([ToTensorV2()])

full_ds = GlacierDataset(CONFIG['base_dir'], transform=train_transform)
val_len = int(len(full_ds)*0.2)
train_ds, val_ds = random_split(full_ds, [len(full_ds)-val_len, val_len], generator=torch.Generator().manual_seed(42))
val_ds.dataset.transform = val_transform

train_loader = DataLoader(Wrapper(train_ds, train_transform), batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(Wrapper(val_ds, val_transform), batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

# ==========================================
# 3. ARCHITECTURE: EFFICIENTNET-V2 SMALL
# ==========================================
class EfficientNetV2Encoder(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        # Load V2 Small
        effnet = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)

        # 1. Adapt Stem (3->5 channels)
        original_conv = effnet.features[0][0]
        new_conv = nn.Conv2d(5, 24, kernel_size=3, stride=2, padding=1, bias=False)
        with torch.no_grad():
            new_conv.weight[:, :3] = original_conv.weight
            new_conv.weight[:, 3:] = original_conv.weight[:, :2]
        effnet.features[0][0] = new_conv

        # 2. SNN Conversion & Surgery
        if mode == "SNN":
            self._convert_to_snn(effnet)

        self.features = effnet.features

        # 3. AUTO-DETECT CHANNELS (Robust Method)
        self.channels = self._auto_detect_channels()
        print(f"✅ V2-Small Detected Channels: {self.channels}")

    def _convert_to_snn(self, model):
        for name, module in model.named_children():
            if isinstance(module, (nn.SiLU, nn.ReLU)):
                setattr(model, name, snn.Leaky(
                    beta=CONFIG['beta'], threshold=CONFIG['threshold'],
                    spike_grad=surrogate.fast_sigmoid(slope=CONFIG['slope']),
                    init_hidden=True))
            elif isinstance(module, models.efficientnet.SqueezeExcitation):
                if CONFIG['remove_se']:
                    setattr(model, name, nn.Identity())
                else:
                    self._convert_to_snn(module)
            elif isinstance(module, nn.Sigmoid):
                setattr(model, name, snn.Leaky(
                    beta=CONFIG['beta'], threshold=0.5,
                    spike_grad=surrogate.fast_sigmoid(slope=CONFIG['slope']),
                    init_hidden=True))
            else:
                self._convert_to_snn(module)

    def forward(self, x):
        feats = []
        # Manual Forward based on V2 Small Structure
        # 0: Stem
        x = self.features[0](x)
        x = self.features[1](x) # Stage 1
        feats.append(x) # x0

        x = self.features[2](x) # Stage 2
        feats.append(x) # x1

        x = self.features[3](x) # Stage 3
        feats.append(x) # x2

        # Stage 4 is layers 4 & 5
        x = self.features[4](x)
        x = self.features[5](x)
        feats.append(x) # x3

        # Stage 5 is layers 6 & 7
        x = self.features[6](x)
        x = self.features[7](x)
        feats.append(x) # x4 (Bottleneck)

        return feats

    def _auto_detect_channels(self):
        """Runs a dummy forward pass to get exact channel counts"""
        dummy = torch.randn(1, 5, 512, 512)
        with torch.no_grad():
            feats = self.forward(dummy)
        return [f.shape[1] for f in feats]

class UnifiedDecoder(nn.Module):
    def __init__(self, channels, mode="CNN"):
        super().__init__()
        c0, c1, c2, c3, c4 = channels # Unpack detected channels

        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        def block(in_c, out_c):
            act = snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True)
            return nn.Sequential(nn.Conv2d(in_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), act)

        self.up4 = nn.ConvTranspose2d(c4, c3, 2, stride=2)
        self.dec4 = block(c3+c3, c3)

        self.up3 = nn.ConvTranspose2d(c3, c2, 2, stride=2)
        self.dec3 = block(c2+c2, c2)

        self.up2 = nn.ConvTranspose2d(c2, c1, 2, stride=2)
        self.dec2 = block(c1+c1, c1)

        self.up1 = nn.ConvTranspose2d(c1, c0, 2, stride=2)
        self.dec1 = block(c0+c0, c0)

        self.final_up = nn.ConvTranspose2d(c0, 32, 2, stride=2)
        self.dec_final = block(32, 32)
        self.final = nn.Conv2d(32, 4, 1)

    def forward(self, enc_feats):
        x0, x1, x2, x3, x4 = enc_feats

        u4 = self.up4(x4)
        if u4.shape != x3.shape: u4 = F.interpolate(u4, size=x3.shape[2:])
        d4 = self.dec4(torch.cat([x3, u4], 1))

        u3 = self.up3(d4)
        if u3.shape != x2.shape: u3 = F.interpolate(u3, size=x2.shape[2:])
        d3 = self.dec3(torch.cat([x2, u3], 1))

        u2 = self.up2(d3)
        if u2.shape != x1.shape: u2 = F.interpolate(u2, size=x1.shape[2:])
        d2 = self.dec2(torch.cat([x1, u2], 1))

        u1 = self.up1(d2)
        if u1.shape != x0.shape: u1 = F.interpolate(u1, size=x0.shape[2:])
        d1 = self.dec1(torch.cat([x0, u1], 1))

        out = self.dec_final(self.final_up(d1))
        return self.final(out)

class UnifiedUNet(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        self.mode = mode
        print(f"⏳ Initializing EffNet-V2-Small {mode} (No-SE={CONFIG['remove_se']})...")
        self.encoder = EfficientNetV2Encoder(mode)
        self.decoder = UnifiedDecoder(self.encoder.channels, mode)

    def forward(self, x):
        if self.mode == "SNN":
            spk_rec = []
            for step in range(CONFIG['time_steps']):
                enc_feats = self.encoder(x)
                out = self.decoder(enc_feats)
                spk_rec.append(out)
            return torch.stack(spk_rec).mean(0)
        else:
            enc_feats = self.encoder(x)
            return self.decoder(enc_feats)

# ==========================================
# 4. TRAINING ENGINE
# ==========================================
def manual_reset(model):
    for m in model.modules():
        if hasattr(m, "reset_mem"): m.reset_mem()

def save_vis(history, sample_vis, epoch, mode):
    plt.figure(figsize=(10, 5))
    plt.subplot(1,2,1); plt.plot(history['loss']); plt.title(f"{mode} Loss")
    plt.subplot(1,2,2); plt.plot(history['mcc']); plt.title(f"{mode} MCC")
    plt.savefig(f"{CONFIG['project_dir']}/{mode}_history.png"); plt.close()

    img, gt, pred = sample_vis
    rgb = img[[3,2,1]].transpose(1,2,0)
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-6)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(rgb); plt.title("Input")
    plt.subplot(1, 3, 2); plt.imshow(gt, cmap='nipy_spectral', interpolation='nearest'); plt.title("GT")
    plt.subplot(1, 3, 3); plt.imshow(pred, cmap='nipy_spectral', interpolation='nearest'); plt.title(f"{mode} Pred")
    plt.savefig(f"{CONFIG['project_dir']}/{mode}_sample.png"); plt.close()

def run_training(phase_name):
    torch.cuda.empty_cache()
    gc.collect()
    print(f"\n🚀 STARTING PHASE: {phase_name}")

    model = UnifiedUNet(mode=phase_name).to(CONFIG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])

    weights = torch.tensor([0.2, 1.0, 1.0, 3.0]).to(CONFIG['device'])
    criterion = nn.CrossEntropyLoss(weight=weights)
    scaler = GradScaler()

    best_mcc = -1.0
    history = {'loss': [], 'mcc': []}

    for epoch in range(CONFIG['epochs']):
        model.train()
        run_loss = 0

        loop = tqdm(train_loader, desc=f"{phase_name} Ep {epoch+1}")
        for imgs, masks in loop:
            imgs, masks = imgs.to(CONFIG['device']), masks.to(CONFIG['device'])

            if phase_name == "SNN": manual_reset(model)

            optimizer.zero_grad()

            if phase_name == "CNN":
                with autocast():
                    out = model(imgs)
                    loss = criterion(out, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(imgs)
                loss = criterion(out, masks)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            scheduler.step()
            run_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        # Val
        model.eval()
        preds, targets = [], []
        sample_vis = None
        with torch.no_grad():
            for i, (imgs, masks) in enumerate(val_loader):
                imgs = imgs.to(CONFIG['device'])
                if phase_name == "SNN": manual_reset(model)

                if phase_name == "CNN":
                    with autocast(): out = model(imgs)
                else:
                    out = model(imgs)

                preds.append(out.argmax(1).cpu())
                targets.append(masks.cpu())
                if i==0: sample_vis = (imgs[0].cpu().numpy(), masks[0].cpu().numpy(), preds[-1][0].numpy())

        mcc = matthews_corrcoef(torch.cat(targets).numpy().flatten(), torch.cat(preds).numpy().flatten())
        history['mcc'].append(mcc)
        history['loss'].append(run_loss/len(train_loader))

        print(f"   ✅ Val MCC: {mcc:.4f}")
        save_vis(history, sample_vis, epoch+1, phase_name)

        if mcc > best_mcc:
            best_mcc = mcc
            torch.save(model.state_dict(), f"{CONFIG['project_dir']}/best_{phase_name}_EffNetV2.pth")

    print(f"🏁 {phase_name} Finished. Best MCC: {best_mcc:.4f}")

if __name__ == "__main__":
    run_training("CNN")
    run_training("SNN")


🚀 STARTING PHASE: CNN
⏳ Initializing EffNet-V2-Small CNN (No-SE=True)...
✅ V2-Small Detected Channels: [24, 48, 64, 160, 1280]


/tmp/ipython-input-2263341684.py:292: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
CNN Ep 1:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-2263341684.py:310: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 1: 100%|██████████| 10/10 [00:09<00:00,  1.10it/s, loss=1.4]
/tmp/ipython-input-2263341684.py:337: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)


   ✅ Val MCC: -0.0081


CNN Ep 2:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-2263341684.py:310: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 2: 100%|██████████| 10/10 [00:03<00:00,  2.87it/s, loss=1.34]
/tmp/ipython-input-2263341684.py:337: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)


   ✅ Val MCC: -0.0079


...

   ✅ Val MCC: 0.5250


SNN Ep 39: 100%|██████████| 10/10 [00:14<00:00,  1.41s/it, loss=0.577]


   ✅ Val MCC: 0.4734


SNN Ep 40: 100%|██████████| 10/10 [00:13<00:00,  1.40s/it, loss=0.66]


   ✅ Val MCC: 0.5141
🏁 SNN Finished. Best MCC: 0.5250
